# Journal Entries with Labelled Emotions
- > https://www.kaggle.com/datasets/madhavmalhotra/journal-entries-with-labelled-emotions/data
- "Lemotif: An Affective Visual Journal Using Deep Neural Networks" 논문에서 발취한 데이터셋
  - > https://arxiv.org/abs/1903.07766
- Mechanical Turk에서 500명의 응답자를 대상으로 설문조사를 실시하여 수집됨.

In [3]:
# 모듈 설치
!pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------- ----- 1.3/1.5 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("madhavmalhotra/journal-entries-with-labelled-emotions")

# 다운로드된 데이터셋 파일이 저장된 경로 출력
print("Path to dataset files:", path)

Path to dataset files: C:\Users\wooll\.cache\kagglehub\datasets\madhavmalhotra\journal-entries-with-labelled-emotions\versions\1


In [6]:
nltk.download('punkt', quiet=True) # 'punkt' 토크나이저 다운로드 - 문장을 단어 단위로 분할하는 데 사용
nltk.download('stopwords', quiet=True) # 'stopwords' 불용어 (copus) 데이터 다운로드
nltk.download('wordnet', quiet=True) # 'wordnet' 사전 데이터 다운로드 - 어휘 의미 네트워크 사용하기 위함 (동의어, 반의어 조회 등 의미 기반 처리에 사용)

True

In [7]:
df = pd.read_csv(r'C:\Users\wooll\OneDrive\문서\GitHub\-\dataset\data.csv')

In [8]:
def cleantext(text):
    # 입력값을 문자열로 변환하고 모두 소문자로 통일
    text = str(text).lower()
    
    # 감정 분류용 키워드 사전 정의 
    emotionkeywrds = {
        'happy': ['happy', 'happiness', 'joy', 'joyful', 'pleased', 'excitement', 'excited', 'glad'],
        'sad': ['sad', 'sadness', 'unhappy', 'depressed', 'depression', 'upset', 'disappointing', 'disappointed', 
                'down', 'low', 'blue', 'sorrow', 'grief', 'gloomy','broke'],
        'angry': ['angry', 'anger', 'furious', 'mad', 'rage', 'outrage', 'annoyed', 'irritated', 'frustrated'],
        'anxious': ['anxious', 'anxiety', 'worried', 'worry', 'nervous', 'tense', 'stress', 'stressed', 'fear', 'afraid'],
        'calm': ['calm', 'peaceful', 'relaxed', 'tranquil', 'serene'],
        'proud': ['proud', 'pride', 'accomplished', 'achievement', 'success', 'successful'],
        'disappointed': ['disappointed', 'disappointment', 'letdown', 'failed', 'failure', 'fail', 'less than expected']
    }
    
    # 모든 감정 키워드를 하나의 집합으로 합치기 
    allemokeywords = set()
    for words in emotionkeywrds.values():
        allemokeywords.update(words)
    
    # 알파벳, 공백, 느낌표/물음표/온점/쉼표만 남기고 나머지 문자 제거 
    text = re.sub(r'[^a-zA-Z\s!?.,]', '', text)
    
    # nltk 불용어 목록 불러오기 
    stop_words = set(stopwords.words('english'))

    # 의미를 뒤집는 부정어 집합 정의 (불용어 제거에서 제외)
    negation_words = {'not', 'no', 'nor', 'neither', 'never', 'none', 'barely', 'hardly', 'scarcely', 'doesnt', 'didnt', 'wasnt', 'isnt', 'arent', 'couldnt', 'shouldnt', 'wouldnt'}
    
    # 축약형 단어 집합 정의 (불용어에 추가)
    contractions = {'arent', 'cant', 'couldnt', 'didnt', 'doesnt', 'dont', 
                    'hadnt', 'havent', 'shouldnt', 'wouldnt', 'youve',
                    'youre', 'wont', 'werent', 'weve', 'wed', 'theyre', 'im'}
    
    # 기본 불용어에 축약형 단어 추가 
    stop_words.update(contractions)

    # 부정어와 감정 키워드는 제거 대상에서 제외함
    stop_words = stop_words - negation_words - allemokeywords
    
    # 토큰화 후 남는 단어만 다시 합치기 
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [9]:
# 처리 시작 메시지 출력
print("Cleaning text...")

# df의 'Answer' 컬럼에 있는 텍스트를 cleantext 함수로 정제하여
# 새로운 'cleantext' 컬럼으로 추가 
df['cleantext'] = df['Answer'].apply(cleantext)

# Identify emotion columns 
# 감정 분석 관련 컬럼만 골라내기 
emotionalcols = [col for col in df.columns if '.f1.' in col and '.raw' in col]

# 추출된 컬럼명에서 실제 감정 이름 추출 
emotion_names = [col.split('.f1.')[1].split('.raw')[0] for col in emotionalcols]

Cleaning text...


In [10]:
# 분포 출력 시작 알림
print("Class distribution for emotions:")

# 클래스 가중치 저장을 위한 빈 딕셔너리 생성
class_weights = {}

# 각 감정 컬럼에 대해 반복
for i, col in enumerate(emotionalcols):
    # 해당 칼럼에 대해 'positive(긍정)' 로 표시된 샘플 수 집계 (컬럼 1에 들어있는 행의 합)
    pos_count = df[col].sum()

    # 전체 샘플 수 
    total = len(df)

    # 감정별 긍정 인스턴스 수와 비율 출력
    print(f"{emotion_names[i]}: {pos_count} positive instances ({pos_count/total*100:.2f}%)")
    
    # 클래스 불균형 보정용 가중치 계산 
    # 긍정 샘플이 적을수록 큰 가중치 부여 
    if pos_count > 0:
        weight = total / (2 * pos_count)
        class_weights[emotion_names[i]] = weight
    else:
        # 해당 감정에 긍정 샘플이 아예 없으면 기본 가중치 1.0 할당 
        class_weights[emotion_names[i]] = 1.0

Class distribution for emotions:
afraid: 18 positive instances (1.22%)
angry: 28 positive instances (1.90%)
anxious: 125 positive instances (8.49%)
ashamed: 17 positive instances (1.15%)
awkward: 15 positive instances (1.02%)
bored: 49 positive instances (3.33%)
calm: 368 positive instances (24.98%)
confused: 28 positive instances (1.90%)
disgusted: 22 positive instances (1.49%)
excited: 251 positive instances (17.04%)
frustrated: 141 positive instances (9.57%)
happy: 730 positive instances (49.56%)
jealous: 3 positive instances (0.20%)
nostalgic: 61 positive instances (4.14%)
proud: 337 positive instances (22.88%)
sad: 43 positive instances (2.92%)
satisfied: 591 positive instances (40.12%)
surprised: 64 positive instances (4.34%)


1. 빈도가 높은 상위 감정들
    - happy (49.6%), satisfied (40.1%), calm (25.0%), excited (17.0%), proud (22.9%), frustrated (9.6%), anxious (8.5%), nostalgic (4.1%), surprised (4.3%), bored (3.3%), sad (2.9%)
2. 매우 희소한 감정들
    - jealous (0.20%), awkward (1.02%), ashamed (1.15%), afraid (1.22%), disgusted (1.49%), confused (1.90%), angry (1.90%)

-> 대부분 긍정적인 감정들이 높은 비율을 차지함. 

In [11]:
# 학습을 위한 특성과 타깃 분류 

# X: 입력 (전처리된 텍스트 데이터)
X = df['cleantext']

# y: 예측해야할 감정 레이블
y = df[emotionalcols]

In [ ]:
# 벡터화 시작 알림 출력
print("Vectorizing text...")

# TF-IDF 벡터라이저 객체 생성 
# 각 토큰의 중요도를 문서 빈도 반비례로 계산함. 
# TF: 한 문서에서 특정 단어가 등장한 횟수 
# IDF: 전체 문서 집합에서 자주 나오는 가중치를 낮추기 위한 반비례 스코어 
# TF-IDF는 이 두개를 곱한 것. 
# n-그램: 텍스트나 말뭉치에서 연속으로 등장하는 n개의 항목(단어나 문자)을 하나의 단위로 보는 기법
# n=1 (Unigram): 한 번에 한 단어씩 끊어서 보는 단위
# n=2 (Bigram): 두 단어씩 묶어서 보는 단위
# n=3 (Trigram): 세 단어씩 묶어서 보는 단위 
vectorizer = TfidfVectorizer(
    max_features=10000, # TF-IDF 행렬의 최대 피처 
    min_df=2, # 전체 문서 대비 등장 빈도가 2회 미만인 단어 무시
    max_df=0.95, # 전체 문서의 95% 이상 등장하는 단어는 무시함. 
    ngram_range=(1, 3), # 연속된 세 단어까지 피처로 추출함. 
    sublinear_tf=True # tf를 1+log(tf) 형태로 스케일링 
)

# fit: 입력(X)을 바탕으로 어휘와 IDF 통계 학습
# transform: 학습된 어휘와 IDF를 사용해 각 문서를 TF-IDF 벡터로 반환
# 두 과정을 한 번에 수행하여 희소 행렬 반환 (행: 문서, 열: n-그램)
X_vec = vectorizer.fit_transform(X)

Vectorizing text...


- 위의 파라미터에 대한 근거도 작성 필요

In [13]:
# 학습/테스트셋 분류
X_train, X_test, y_train, y_test = train_test_split(
  X_vec, 
  y, 
  test_size=0.2, 
  random_state=42, 
  stratify=(
    df[emotionalcols[0]] # 첫 번째 감정 컬럼
    if df[emotionalcols[0]].sum() > 10 # 그 칼럼에서 1인(긍정) 샘플이 10개 초과일 때만 stratify
    else None # 그렇지 않으면 startify 하지 않음 
    ) # 
  )

print("Training model...")

# 기본 모델: 로지스틱 회귀
base_model = LogisticRegression(
    C=1.0,
    class_weight='balanced', # 불균형 데이터에 자동으로 가중치 부여 
    max_iter=1000,
    solver='liblinear',
    random_state=42
)

# 다중 출력 분류기 래핑: 한 번에 여러 감정(다중 레이블)을 독립적으로 예측할 수 있도록 확장
model = MultiOutputClassifier(base_model)
model.fit(X_train, y_train)

Training model...


MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   max_iter=1000,
                                                   random_state=42,
                                                   solver='liblinear'))

- 위의 파라미터에 대한 근거 필요

In [14]:
# 평가 시작 
print("Evaluating model...")

# 테스트셋에 대한 예측 수행
y_pred = model.predict(X_test)

# 성능 지표 출력
print(classification_report(y_test, y_pred, target_names=emotion_names, zero_division=0))

Evaluating model...
              precision    recall  f1-score   support

      afraid       0.00      0.00      0.00         4
       angry       0.00      0.00      0.00         3
     anxious       0.44      0.33      0.38        21
     ashamed       0.00      0.00      0.00         2
     awkward       0.00      0.00      0.00         2
       bored       0.50      0.17      0.25         6
        calm       0.45      0.43      0.44        76
    confused       0.00      0.00      0.00         5
   disgusted       0.00      0.00      0.00         2
     excited       0.12      0.09      0.11        53
  frustrated       0.54      0.28      0.37        25
       happy       0.77      0.61      0.68       170
     jealous       0.00      0.00      0.00         0
   nostalgic       0.17      0.08      0.11        12
       proud       0.48      0.42      0.45        79
         sad       0.00      0.00      0.00         5
   satisfied       0.53      0.61      0.57       120
   surp

1. 높은 성능
- happy: precision 0.77, recall 0.61, f1 0.68 (support 170)
- satisfied: precision 0.53, recall 0.61, f1 0.57 (support 120)
- calm: precision 0.45, recall 0.43, f1 0.44 (support 76)
- proud: precision 0.48, recall 0.42, f1 0.45 (support 79)
- frustrated: precision 0.54, recall 0.28, f1 0.37 (support 25)

2. 중간 수준 성능
- anxious: precision 0.44, recall 0.33, f1 0.38 (support 21)
- bored, nostalgic, surprised, excited 등은 support가 653 사이로 적당하지만 f1-score는 0.10.25 수준

3. 희소 클래스 (매우 낮거나 예측 거의 못함)
- angry, ashamed, awkward, confused, disgusted, sad, afraid 등 소수 샘플(≤6건)
- precision=0.00, recall=0.00, f1=0.00 → 학습·예측 불가
- jealous는 test에 전혀 출현하지 않아(support=0) 평가 불가

In [16]:
# 감정별 임계값(threshold)
def get_emotion_thresholds():
    # 빈 딕셔너리 생성
    thresholds = {}
    
    # class_weight에 저장된 (emotion, weight) 쌍 조회
    for emotion, weight in class_weights.items():

        # weight가 5보다 크면, 예측 확률 기준을 0.2로 낮춤 -> 학습된 모델이 긍정(1)으로 더 쉽게 판단하게 함. 
        if weight > 5:
            thresholds[emotion] = 0.2
        
        # weight가 2보다 크면, 중간 수준의 기준 부여 
        elif weight > 2:
            thresholds[emotion] = 0.3
        # 그 외(자주 등장하는)는 기본 수준 부여  
        else:
            thresholds[emotion] = 0.4
    
    # 특정 감정에 대해서는 수동으로 override 함 
    # 'happy', 'calm', 'proud' 는 모델이 좀 더 엄격하게 판단하도록 함.
    overrides = {
        'happy': 0.55,
        'calm': 0.55,
        'proud': 0.5
    }
    
    # thresholds 딕서녀리에 overrides 값 반영
    thresholds.update(overrides)

    # 최종 임계값 딕셔너리 반환 
    return thresholds

- 임계값 설정 기준 서술필요

In [17]:
# 다중 감정 예측
def predict_emotions_advanced(user_input, vectorizer, model, emotion_names):
    # 1. 임계값 불러오기 
    emotion_thresholds = get_emotion_thresholds()
    # 정해지지않은 임계값에 대비해 기본값 0.35로 설정
    default_threshold = 0.35

    # 2. 입력 전처리 
    # 2-1. 소문자화, 불용어 제거 등 정제 
    cleaned = cleantext(user_input)
    # 2-2. 이미 학습된 vectorizer로 TF-IDF 벡터 변환
    vectorized = vectorizer.transform([cleaned])
    # 2-3. 토큰 단위로 쪼개서 input_words 집합 생성 
    input_words = set(cleaned.split())
    
    # 3. 키워드 매칭
    # 감정별 주요 키워드 사전 
    emotionkeywrds = {
        'happy': ['happy', 'joy', 'joyful', 'pleased', 'excitement', 'excited', 'glad'],
        'sad': ['sad', 'unhappy', 'depressed', 'depression', 'upset', 'disappointing', 'disappointed', 
                'down', 'low', 'blue', 'sorrow', 'grief','not','less','broke','reject'],
        'angry': ['angry', 'anger', 'furious', 'mad', 'rage', 'outrage', 'annoyed', 'irritated'],
        'anxious': ['anxious', 'anxiety', 'worried', 'worry', 'nervous', 'tense', 'stress', 'stressed'],
        'fear': ['fear', 'afraid', 'scared', 'frightened', 'terrified'],
        'surprised': ['surprised', 'surprise', 'shocking', 'shocked', 'unexpected'],
        'disgusted': ['disgust', 'disgusted', 'gross', 'revolting'],
        'calm': ['calm', 'peaceful', 'relaxed', 'tranquil', 'serene'],
        'proud': ['proud', 'pride', 'accomplished', 'achievement', 'success', 'successful'],
        'disappointed': ['disappointed', 'disappointment', 'letdown', 'failed', 'failure', 'fail', 'less than expected']
    }
    
    keywords_found = {}

    # 감정별로 정의된 키워드 리스트 중 입력 문장에 등장하는 단어를 찾아 저장함. 
    for emotion, keywords in emotionkeywrds.items():
        for word in keywords:
            if word in cleaned:
                if emotion not in keywords_found:
                    keywords_found[emotion] = []
                keywords_found[emotion].append(word)
    
    # 4. 모델 예측 & 임계값 비교 
    emotions_detected = []
    
    try:
        for i, estimator in enumerate(model.estimators_):
            emotion = emotion_names[i]
            threshold = emotion_thresholds.get(emotion, default_threshold)
            proba = estimator.predict_proba(vectorized)[0][1]
            # 키워드가 발견된 감정은 임계값을 0.7배 낮춰 더 쉽게 검출하도록 함
            if emotion in keywords_found:
                threshold *= 0.7
            # 확률이 임계값을 넘으면 (감정, 확률) 쌍을 emotions_detected에 추가 
            if proba > threshold:
                emotions_detected.append((emotion, float(proba)))
    # 예외 처리 
    except:
        raw_predictions = model.predict(vectorized)[0]
        for i, val in enumerate(raw_predictions):
            emotion = emotion_names[i]
            if val == 1:
                emotions_detected.append((emotion, 0.9))
    
    # 결과 정렬 
    emotions_detected.sort(key=lambda x: x[1], reverse=True)
    
    # 특수 규칙 적용 -> 일반 모델이 예측하기 어려운 부정적 결과에 대한 실망을 키워드 기반으로 보강함. 
    # 'result'와 부정어('less', 'fail', not)가 함께 있을 때 
    if "result" in cleaned and ("less" in cleaned or "fail" in cleaned or "not" in cleaned):
        # 아직 'sad'나 'disapointed'가 검출되지 않았다면 'disapointed'를 확률 0.75로 강제 추가 -> 사용자가 결과가 좋지 않다는 뉘앙스를 드러낸 것으로 간주 
        if not any(e[0] in ["sad", "disappointed"] for e in emotions_detected):
            emotions_detected.append(("disappointed", 0.75))
    
    return emotions_detected

In [ ]:
test = input()

predicted_emotions = predict_emotions_advanced(test, vectorizer, model, emotion_names)
    
print("\nPredicted Emotions for text:")
print(f"Text: '{test}'")
if predicted_emotions:
    for emotion, prob in predicted_emotions:
        print(f"{emotion}: {prob:.4f}")
else:
    print("No strong emotions detected.")